# Coordinate Reference Systems and Affine Transformations

## What is a coordinate reference system (CRS)?

Coordinate reference systems (CRS) are an integral part to **georeferencing** your data - or placing your data in space.  Without georeferencing raster datasets are just pretty images that could have been taken anywhere in the world.

The CRS works together with the map projection to define how your dataset describes where each pixel is.

There is a ton to know about CRSs and map projections.  A lot of this information falls under the purview of **geodesy** - the science of understanding and measureing the earth's shape.  For now we will just hit on a few major points.

### Some Quick Points about CRS

**Number 1 - Two types of units**

The units of your reference system probably fit into one of two categories:
* units of degrees (ex. latitude and longitude) - geographic CRS
* units of distance (ex. meters, kilometers, miles, ...) - projected CRS

**Number 2 - EPSG codes**

One common way to notate coordinate reference systems and map projections is with EPSG codes.  These codes assign an integer number to many of the common map projection/crs combinations.

## EPSG:4326

EPSG:4326 in my experience is the most common map projection.  It is just regular latitude and longitude coordiantes.

## UTM Grids

The Universal Transverse Mercator (UTM) projected coordinate system that covers the entire earth.  The system is broken up into 60 zones, each 6 degrees longitude wide, that span from the south to the north pole.

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
# UTM grid zones
Image(url= "https://gisgeography.com/wp-content/uploads/2016/05/UTM-Zones-Globe-2.png", width=330)

Notice in the image how the area covered by each zone is widest at the equator and gets skinnier towards the poles.

### Easting and Northing

Instead of latitude and longitude the UTM grid system uses a northing and an easting.
- Eastings - measured from the central meridian, which has a value of 500,000m
- Northings - measured from the from the equator, which has a value of 0 for the northern hemisphere and 10,000,000m for the southern hemisphere

Benefits of UTM:
- uses a square grid -> constant distance relationship anywhere on the map
- All values of eastings/northings are positive, so there are no negative numbers nor is there an East-West designation

In [3]:
Image(url= "https://www.maptools.com/images/28ad74e.png", width=360)

The x direction should only range from about 0 to 1,000,000 where as the y direction could range from 0 to about 10,000,000 at a maximum.

### EPSG codes for UTM grids

Each grid has its own EPSG code, meaning there are 120 total EPSG codes total for UTM grids.  Luckily there is an order to how these codes are assigned.


* northern hemisphere - 326{zone number}
* southern hemisphere - 327(zone number}


To quickly reference UTM grid zone numbers I use the image from [this website](http://www.dmap.co.uk/utmworld.htm) as a reference:

In [4]:
# UTM grid zones
Image(url= "http://www.dmap.co.uk/utmworld.gif")

**Example**

So the EPSG code for the UTM zone covering most of Ireland is `32629`.  

If you want another place to check this is correct you can go to epsg.io and look at the map.  https://epsg.io/32629 does indeed go through Ireland, so we can be satisfied with this conclusion.

## Back to our dataset

So to figure out where a particular point is on the earth we need two things:
1. the EPSG code to tell us which grid we are looking at
2. our coordinates

### Example with AVIRIS

Since our AVIRIS data uses UTM grids our coordinates will be an easting and a northing.

In [5]:
import rasterio

In [6]:
filepath = '../input_data/f150131t01p00r10_refl/f150131t01p00r10_h2o_v1'

In [7]:
with rasterio.open(filepath, 'r') as src:
    bbox = src.bounds
    src_crs = src.crs
    
print('crs is ', src_crs)
print('bbox is ', bbox)

crs is  EPSG:32610
bbox is  BoundingBox(left=278669.3124746986, bottom=3817906.830412161, right=339811.7357139058, top=3967305.2)


Looking at the output of the above cell we see that the EPSG code is `32613`.  The 6 in this code tells us that we are looking at the northern hemisphere, and the `13` tells us that we are looking at UTM zone 13.

Depending on why you want these coordinates the numbers still might not be very useful unless we can get them to latitude and longitude.  To seem some examples of converting points from UTM to EPSG:4326 (lat/lon) look at the "Reprojecting to EPSG4326" notebook.

## Affine Transformations

Affine transformations are the way that we move from row, column notation (pixel space) to the notation of a coordinate reference system (usually either latitude & longitude or easting & northing)

My favorite affine transformation article is [this one](https://www.perrygeo.com/python-affine-transforms.html) written by Matthew Perry.

Here is what an affine tranformation looks like:

In [8]:
with rasterio.open(filepath, 'r') as src:
    print(src.transform)

| 16.64,-2.93, 303895.60|
|-2.93,-16.64, 3967305.20|
| 0.00, 0.00, 1.00|


Where each number means something different:

| | |  |
|---|---|---|
|  a |  b | c  |
|  d | e  | f  |
| 0 | 0 | 1 |


* a = width of a pixel in units of the crs
* b = row rotation (typically zero)
* c = x-coordinate of the upper-left corner of the upper-left pixel
* d = column rotation (typically zero)
* e = height of a pixel in units of the crs (typically negative)
* f = y-coordinate of the of the upper-left corner of the upper-left pixel

The three numbers in the bottom row are always 0, 0, 1 (since we are working on a 2 dimensional plane).

### Using the affine to get the coordinates of a particular grid cell

In [9]:
# Define which pixel we want to convert
row, col = 6, 1601

# Extract the affine transform
with rasterio.open(filepath, 'r') as src:
    affine = src.transform

# Use the transform to convert our input pixel row/column to coordinates
x, y = affine * (col, row)
print('coordinates of my pixel: ', x, y)

coordinates of my pixel:  330523.8369672606 3962506.9591155285


### Using the inverse affine to the the pixel of a coordinate

The affine transformation takes you from the row/column pixel space to your crs. To go in the other direction you use the inverse transform, calculated with the ~ operator.

In [10]:
# Define the coordinates we want to convert
xcoord, ycoord = -128, 34

# Extract the affine transform
with rasterio.open(filepath, 'r') as src:
    affine = src.transform

# Use the transform to convert our input coordinates to pixel row/column
col, row = ~affine * (xcoord, ycoord)
print('row/column of my coordinates: ', row, col)

row/column of my coordinates:  234307.25326448443 23047.6104030659


If you get a negative number here or your pixel row/column are bigger than the height or width of your raster it means you have asked for the pixel location of a point which is not located in your raster.

In [11]:
with rasterio.open(filepath, 'r') as src:
    print('shape of my raster ', src.height, src.width)

shape of my raster  8596 2158
